In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

In [2]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# multi[curation_col]= multi[curation_col].apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

# matches=not_tube
# matches[curation_col]=matches['height'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'7\/165','.44',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))


# string = range_string
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return f'{lower}" - {upper}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return f'{original_val}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

# matches[curation_col]=matches['Min'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]  

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        print(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

# str(round(num, 2))
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers

def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

In [4]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = """Multiple Buckets"""
strategy_version_input=705
attribute_id_input=2426
formatted_attribute = 'Width'

In [5]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [6]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input} 
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()] 

In [7]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df)) 
df = df[df['bucket_id'].isin(buckets)]
height=df
len(df)

17893


17893

In [8]:
df['buckets'].explode().value_counts()[0:50] 

Fixings & Fasteners                        16899
Electrical System                            908
Writing/Design Implements & Accessories       70
Cleaning Tools & Accessories                  13
Cabinet Accessories                            2
Waste / Refuse Bags                            1
Name: buckets, dtype: int64

In [9]:
regex_pattern=r'(.{0,20}width.{0,20})|()'
df['matches']=df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[df['matches'].astype(str)!='[]']))
df[df['matches'].astype(str)!='[]']

28


product_id external_id  \
113    1173679390      831530   
581    1173666559      501148   
587    1173666549      501120   
672    1173666633      501106   
673    1173666630      501104   
5159   1173654794      437737   
9178   1173647689       26497   
10450  1173637820    402662_2   
10451  1173637819    402661_2   
10452  1173637817    402653_2   
10453  1173637816    402652_2   
10454  1173637815    402651_2   
10602  1173666631      501112   
11438  1173654793      437735   
11439  1173654792      437736   
11687  1173637839      402641   
11691  1173637818     40266_2   
11721  1173666632      501164   
11947  1173661361      131043   
14258  1173666554      501128   
14530  1173666531      501136   
15805  1173647690       26722   
16120  1173654801      437728   
16121  1173654800      437733   
16622  1173654796      437734   
16639  1173637814     40265_2   
16705  1173637813     40264_2   
17554  1173654797      437732   

                                                 product_name  \
113                              Water Removable Paint Marker   
581                                  #48 Key-Drive Hose Clamp   
587                                  #20 Key-Drive Hose Clamp   
672                              #6 Key-Drive Mini Hose Clamp   
673                              #4 Key-Drive Mini Hose Clamp   
5159      #10 x 4" Torx Drive Flat Head Power-Thread II Screw   
9178                  3" Dipped Steel Wiring and Tubing Clamp   
10450              Insulated Stainless Steel Clamp Assortment   
10451              Insulated Stainless Steel Clamp Assortment   
10452                     Insulated Aluminum Clamp Assortment   
10453                        Insulated Steel Clamp Assortment   
10454                        Insulated Steel Clamp Assortment   
10602                                #12 Key-Drive Hose Clamp   
11438   #8 x 2-1/2 Torx Drive Flat Head Power-Thread II Screw   
11439  #10 x 3-1/8 Torx Drive Flat Head Power-Thread II Screw   
11687            Double Tube Plastisol Steel Clamp Assortment   
11691                     Insulated Aluminum Clamp Assortment   
11721                                #64 Key-Drive Hose Clamp   
11947                                       Fender Apron Clip   
14258                                #28 Key-Drive Hose Clamp   
14530                                #36 Key-Drive Hose Clamp   
15805          3/8 Insulated Aluminum Wiring and Tubing Clamp   
16120   #6 x 1-1/4 Torx Drive Flat Head Power-Thread II Screw   
16121   #8 x 1-3/4 Torx Drive Flat Head Power-Thread II Screw   
16622      #8 x 2" Torx Drive Flat Head Power-Thread II Screw   
16639                           Dipped Steel Clamp Assortment   
16705                           Dipped Steel Clamp Assortment   
17554   #8 x 1-1/2 Torx Drive Flat Head Power-Thread II Screw   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [14]:
regex_pattern=r'(M\d+)|()'
df['matches']=df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[df['matches'].astype(str)!='[]']))
df[df['matches'].astype(str)!='[]']

263


product_id external_id  \
667    1173662431       13212   
685    1173662402       13214   
721    1173661967       13066   
727    1173661953       13065   
728    1173661951       13327   
745    1173661835       13964   
746    1173661834       13963   
763    1173661796       13611   
766    1173661793       13556   
767    1173661791       13473   
768    1173661772       13552   
777    1173661777       13562   
779    1173661775       13613   
781    1173661766       13615   
783    1173661759       13486   
784    1173661758       13558   
785    1173661757       13475   
787    1173661755       13471   
794    1173661735       13568   
795    1173661732       13515   
809    1173661699       13601   
810    1173661698       13551   
815    1173661683       13481   
816    1173661679       13610   
825    1173661659       10142   
831    1173661650       10920   
835    1173661645       13554   
843    1173661630       10159   
844    1173661628       13567   
852    1173661617       10178   
854    1173661614       10143   
863    1173661592       13608   
869    1173661584       13603   
910    1173661502       13844   
915    1173661944       13032   
921    1173661931       13082   
922    1173661930       13064   
927    1173661922       13665   
931    1173661918       13009   
944    1173661892       13047   
949    1173661880       13054   
951    1173661877       13304   
967    1173661022       13874   
971    1173661005      131328   
1109   1173661398      131412   
1112   1173661386      131383   
1113   1173661385      131382   
1116   1173661374      131345   
1118   1173661371      131211   
1123   1173661363      131046   
1124   1173661362      131045   
1347   1173660960       12799   
1354   1173660952       13588   
1357   1173660944       13466   
1360   1173660941       13729   
1363   1173660936       13712   
1365   1173660934       13467   
1368   1173660929       13468   
1370   1173660927       13728   
1374   1173660922       13469   
5661   1173652083       10656   
5687   1173651852      465912   
5688   1173651849      466012   
5693   1173651821      465824   
5699   1173651805      465910   
5703   1173651800      465814   
5707   1173651786      465820   
5713   1173651779      465806   
5724   1173651762      465808   
5725   1173651761      465812   
5732   1173651745      465810   
9426   1173662371       13680   
9440   1173661477      131359   
9441   1173661476      131339   
9442   1173661474      131320   
9457   1173661424       13998   
9460   1173661417       13991   
9462   1173661409       13855   
9466   1173661403       13847   
9474   1173661326      131009   
9480   1173661318      131001   
9482   1173661315       13770   
9488   1173661308       13769   
9497   1173661293       13707   
9498   1173661289       12855   
9504   1173661280       10218   
9505   1173661279       10229   
9506   1173661277       12589   
9511   1173661268       10213   
9512   1173661263       12904   
9516   1173661254       12942   
9523   1173661242       13631   
9539   1173661217       10219   
9541   1173661190       10209   
9546   1173661174       13629   
9547   1173661182       13630   
9550   1173661165       13899   
9554   1173661161       13895   
9559   1173661156       13889   
9561   1173661154       13887   
9569   1173661146      131358   
9574   1173661139      131340   
9575   1173661138      131332   
9587   1173661115       13655   
9588   1173661114       13654   
9604   1173661096       12173   
9607   1173661093       13653   
9615   1173661079       13658   
9617   1173661077       12157   
9623   1173661069       13657   
9625   1173661066       12176   
9628   1173661061       12177   
9629   1173661060       12178   
9635   1173661053       12161   
9641   1173661047       13646   
9644   1173661044       13642   
9646   1173661042       13640   
9652   1173662387       13745   
9663   1173662351       13826   
9665   1173662348      131431   
9674   1173662

# General

In [9]:

# regex_pattern_general=r'(?i)(\b\d+\.\d*.?\"\sH)|(\b\d+.?\"\sH)|(\b\d+\.\d*.?\"\sH)|(\d*\s?\-\d*\/\d*\\\"\sH)|(\d*\s\-?\d*\/\d*\\\"\sH)|(\d+\.?\d*''\sH)|()'
# # regex_pattern_general=r'(?i)(\b\d+\.\d*.?\"\s?H)|(\b\d+.?\"\s?H)|(\b\d+\.\d*.?\"\s?H)|(\d*\s?\-\d*\/\d*\\\"\s?H)|(\d*\s\-?\d*\/\d*\\\"\s?H)|(\d+\.?\d*''\s?H)|()'
# height['matches'] = height['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))
# height['name_matches'] = height['product_name'].apply(lambda x: re_extract(regex_pattern_general, x))
# height['matches']= height['matches'].apply(lambda x: sorted(x, key=natural_key))
# print(len(height))
# print(len(height[height['matches'].astype(str)=='[]']))
# print(len(height[height['matches'].astype(str)!='[]']))

In [10]:
# # 3
# (\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|

# # All
# (?i)(\d\d\.?\d*\s?h)|(\d+\s?\d*\/\d*\s?h)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|(\d+\.?\s?\-?\d*\/?\d*"H)|(height\s?without\s?casters.?\s?\d+\")

# Protective Packing Materials    

In [11]:
packing=height[height['buckets'].astype(str)=='Protective Packing Materials']
len(packing)#variable

0

In [12]:

# not_tube[['length', 'width', 'height']]=not_tube['matches'].str.split("x",expand=True,)

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

# not_tube['length']=not_tube['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# not_tube['width']=not_tube['width'].apply(lambda quote: quote.replace('"', ''))

# regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
# not_tube['length_new']=not_tube['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
# not_tube['width_new']=not_tube['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)

# not_tube['length_new'] = not_tube['length_new'].apply(lambda x: remove_fraction(x))
# not_tube['width_new'] = not_tube['width_new'].apply(lambda x: remove_fraction(x))

# not_tube['length_new'] = not_tube['length_new'].apply(lambda x: remove_frac(x))
# not_tube['width_new'] = not_tube['width_new'].apply(lambda x: remove_frac(x))

# not_tube['length_new']=not_tube['length_new'].astype(float)
# not_tube['width_new']=not_tube['width_new'].astype(float)

# not_tube['Max'] = not_tube[['length_new', 'width_new']].values.max(1)
# not_tube['Min'] = not_tube[['length_new', 'width_new']].values.min(1)


# TWOPLACES = Decimal(10) ** -2
# not_tube['Max']=not_tube['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x)))
# not_tube['Min']=not_tube['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

# envelope['envelope_matches']=envelope['envelope_matches'].str.get(0)
# envelope['envelope_matches']=envelope['envelope_matches'].apply(lambda x: x.lower())


# envelope[['length', 'width']]=envelope['envelope_matches'].str.split("x",expand=True,)

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

# envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# envelope['width']=envelope['width'].apply(lambda quote: quote.replace('"', ''))
# # envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('-',''))
# # envelope['width']=envelope['width'].apply(lambda quote: quote.replace('-',''))


# regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
# envelope['length_new']=envelope['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
# envelope['width_new']=envelope['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)


# envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_fraction(x))
# envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_fraction(x))

# envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_frac(x))
# envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_frac(x))

# Mailing & Packing Boxes

In [74]:
Boxes=height
Boxes=Boxes[Boxes['buckets'].astype(str)=='Mailing & Packing Boxes']
regex_pattern_tube=r'(?i)(tube)|()'
tubes=Boxes
tubes['tube_matches']=tubes['long_desc'].apply(lambda x: re_extract(regex_pattern_tube, x))

not_tube=tubes[tubes['tube_matches'].astype(str)=='[]']
regex_pattern=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|()'              
not_tube['matches']=not_tube['product_name'].apply(lambda x: re_extract(regex_pattern, x))
not_tube=not_tube[not_tube['matches'].astype(str)!='[]']

In [75]:
not_tube

product_id external_id  \
57     434458586  5637421775   
58     434456503  5637421677   
59     434456496  5637421671   
60     434456492  5637421668   
61     434456491  5637421667   
...          ...         ...   
8976   434482628  5637462638   
8979   434472475  5637439657   
8981   434503343  5637484173   
9024   434483422  5637462467   
9044   434492586  5637464360   

                                                                                            product_name  \
57    W.B. Mason Co. Brown Corrugated - Fixed Depth boxes, 17 1/4l x 11 1/4w x 14 1/4h, Brown, 25/Bundle   
58                             W.B. Mason Co. Double Wall Corrugated boxes, 36" x 22" x 22", Kraft, 5/BD   
59                                    W.B. Mason Co. Flat Corrugated boxes, 19" x 12" x 3", Kraft, 25/BD   
60                                        W.B. Mason Co. Double Wall boxes, 18" x 18" x 6", Kraft, 15/BD   
61                                         W.B. Mason Co. Corrugated boxes, 36" x 35" x 40", Kraft, 5/BD   
...                                                                                                  ...   
8976                                                       CORRUGATED BOX-18X13X7 1/2 RSC/GLUE 32ECT J18   
8979               W.B. Mason Co. Reverse Tuck Folding Cartons, 3 1/4" x 15/16" x 3 1/4", White, 1000/CS   
8981                                                   W.B. Mason Co. BOX,CORRUGATED, 18"X18"X16" 20/BDL   
9024                                                       CORR BOX RSC 14 1/16 X 8 X6 200C 4P-1C 110634   
9044                                                       CORRUGATED BOX 22X18X16 RSC 32ECT NPI MEDICAL   

                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
57                                                                                                    Brown Corrugated - Fixed Depth boxes, 17 1/4l x 11 1/4w x 14 1/4h, Brown, 25/Bundle•This listing is for a bundle of 25 boxes•Protect shipping contents with brown, quality, standard strength corrugated shipping boxes•Manufactured from 200 lb. corrugated Kraft; ECT-32 C-Flute•Boxes ship flat to save space   
58                                                                                                                                       Double Wall Corrugated boxes, 36" x 22" x 22", Kraft, 5/BD•This listing is for a bundle of 5 boxes•Use these large boxes to consolidate multiple piece shipments•Protects bulky, hard to pack, irregularly shaped items•Can support the content weight of 101 lbs. - 120 lbs.   
59    Flat Corrugated boxes, 19" x 12" x 3", Kraft, 25/BD•This listing is for a bundle of 25 boxes•The smaller depths on these cartons provide an ideal fit for shallow products•Perfect for shipping apparel, books, picture frames, artwork, mirrors and clothing•Cartons are sold in bundle quantities and ship flat to save on storage space and shipping•Boxes are manufactured from 200#/ECT-32 kraft corrugated   
60                                                   Double Wall boxes, 18" x 18" x 6", Kraft, 15/BD•This listing is for a bundle of 15 boxes•Twice the protection of a standard carton•Manufactured from heavy-duty, doublewall, kraft corrugated•Heavy-duty construction provides greater protection and stacking strength•Cartons are sold in bundle quantities and ship flat to save on storage space and shipping   
61                                                                                                                                              Corrugated boxes, 36" x 35" x 40", Kraft, 5/BD•This listing is for a bundle of 5 boxes•Quality standard strength industrial Corrug

In [60]:
# print(len(not_tube))

In [61]:
# not_tube[0:500]

In [76]:
not_tube['matches']=not_tube['matches'].str.get(0).apply(lambda x: x.lower())
not_tube[['length', 'width', 'height']]=not_tube['matches'].str.split("x",expand=True,)

In [77]:
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'15250','15',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'1232','12',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'15250','15',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'24141680','2',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'16200','16',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'9275','9',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'1832','18',str(measure)))

In [78]:
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure)))
not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))

In [79]:
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))

In [80]:
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
not_tube['height']=not_tube['height'].replace('"', '').apply(lambda quote: quote.replace('"', ''))

In [81]:
regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
not_tube['height_new']=not_tube['height'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
not_tube['height_new'] = not_tube['height_new'].apply(lambda x: remove_fraction(x))

In [82]:
not_tube['height_new'] = not_tube['height_new'].apply(lambda x: remove_frac(x))
not_tube['height_new']=not_tube['height_new'].astype(float)
TWOPLACES = Decimal(10) ** -2
not_tube['height_new']=not_tube['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [83]:
not_tube[curation_col]=not_tube['height_new']
not_tube[curation_col]=not_tube[curation_col].apply(lambda x: str(x)+' in')

In [84]:
not_tube[0:500]

product_id external_id  \
57     434458586  5637421775   
58     434456503  5637421677   
59     434456496  5637421671   
60     434456492  5637421668   
61     434456491  5637421667   
63     434456484  5637421661   
64     434456482  5637421659   
65     434456480  5637421658   
66     704220935  5637549986   
67     704220929  5637549980   
68     704220921  5637549972   
72     704220903  5637549954   
75     698495806  5637549562   
76     698495802  5637549558   
79     698495782  5637549538   
80     698495780  5637549536   
81     698495778  5637549534   
82     698495776  5637549532   
83     698495774  5637549530   
84     698495772  5637549528   
85     698492140  5637549524   
86     698492138  5637549522   
87     698492134  5637549518   
88     698492128  5637549512   
89     698492126  5637549510   
90     698492124  5637549508   
91     698492120  5637549504   
92     698492118  5637549502   
93     698492116  5637549500   
94     698492112  5637549496   
95     698492108  5637549492   
96     698492102  5637549486   
97     698492100  5637549484   
98     698492096  5637549480   
99     698492094  5637549478   
100    698492088  5637549472   
101    698492086  5637549470   
102    698492082  5637549466   
103    698492080  5637549464   
104    698492076  5637549460   
106    698492070  5637549454   
107    698492068  5637549452   
108    698492062  5637549446   
109    698492052  5637549436   
113    675182782  5637544268   
114    675182774  5637544260   
120    675182756  5637544242   
123    675182750  5637544236   
124    675182748  5637544234   
125    675182746  5637544232   
126    675182744  5637544230   
127    675182742  5637544228   
128    675182738  5637544224   
129    675182736  5637544222   
130    675182732  5637544218   
131    675182720  5637544206   
132    675182718  5637544204   
133    675182716  5637544202   
134    675182712  5637544198   
172    434388516  5637233201   
173    434388515  5637233200   
174    434449442  5637404747   
175    675180530  5637543686   
176    675180413  5637543678   
177    675180527  5637543672   
178    675180455  5637543800   
179    675180609  5637543798   
180    675180607  5637543796   
181    675180599  5637543788   
183    675180591  5637543780   
184    675180583  5637543772   
185    675180453  5637543768   
186    675180449  5637543764   
187    675180445  5637543760   
188    675180443  5637543758   
189    675180576  5637543754   
190    675180574  5637543752   
191    675180439  5637543744   
192    675180435  5637543740   
193    675180433  5637543738   
194    675180565  5637543732   
195    675180563  5637543730   
196    675180561  5637543728   
198    675180553  5637543720   
199    675180551  5637543718   
200    675180543  5637543710   
201    675180541  5637543708   
202    675180429  5637543704   
203    675180427  5637543702   
208    434418593  5637337141   
209    434416240  5637337140   
210    434416231  5637337130   
211    434416230  5637337129   
213    434416218  5637337117   
214    434416212  5637337111   
216    434482663  5637462678   
217    434482652  5637462665   
218    434482648  5637462661   
219    434482634  5637462647   
220    434482627  5637462637   
221    434482626  5637462636   
222    680258541  5637462631   
223    434482614  5637462618   
224    434482604  5637462598   
226    434483513  5637462583   
268    434460654  5637425470   
270    434460648  5637425465   
272    434460644  5637425461   
273    434460641  5637425457   
274    434460634  5637425451   
275    434460630  5637425447   
276    434460628  5637425445   
277    434459053  5637425440   
279    434459051  5637425438   
281    434459046  5637425432   
282    434459045  5637425431   
285    434459039  5637425423   
286    434459033  5637425418   
287    434459028  5637425414   
288    434459025  5637425412   
308    434479820  5637459019   
334    434460586  5637425215   
335    434460579  5637425209   
336    434460576  5637425206   
337   

# Envelope

In [26]:
# envelope=height[height['buckets'].astype(str)=='Envelopes/Mailers']
# regex_pattern_general=r'(?i)(roll)|()'
# envelope['envelope_matches']=envelope['product_name'].apply(lambda x: re_extract(regex_pattern_general, x))
# envelope=envelope[envelope['envelope_matches'].astype(str)=='[]']

In [27]:
# regex_pattern=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|()'              
# envelope['matches']=envelope['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# envelope=envelope[envelope['matches'].astype(str)=='[]']

In [28]:
# envelope[0:500] # Waiting on Arv and Ross

In [29]:
# regex_pattern_general=r'(?i)(\d+\s?\-?\d+\/\d+.?\s?x\s?\d+\s?\-?\d+\/\d+)|(\d+\.\d*.?\s?x\s?\d+\.\d*)|(\d+\s?\-?\d+\/\d+.?\s?x\s?\d+\.?\d*)|(\d+\s?.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+)|(\d+.?\d*.?\s?x\s?\d+.?\d*)|(\d+\.?\s?\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|()'    
# envelope['envelope_matches']=envelope['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))
# envelope=envelope[envelope['envelope_matches'].astype(str)!='[]']

In [30]:
# def array(df):
#     df['matches_one']=df['envelope_matches'].apply(lambda x : np.array(x)[0])
#     try:
#         df['matches_two']=df['envelope_matches'].apply(lambda x : np.array(x)[1])
#     except:
#         df['matches_two']='n/a'

In [53]:
# def drop_characters(data):
#     data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))

In [54]:
# array(envelope)

In [55]:
# envelope['match_one'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[0])

In [56]:
# try:
#     envelope['matches'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[1])
# except:
#     envelope['matches'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[0])

In [57]:
# envelope[0:500]

In [58]:
# # double=envelope[envelope['external_id'].astype(str)=='5637503643']
# double=envelope[0:1]
# double

In [59]:
# double['n_matches']=double['envelope_matches'].apply(lambda x : np.array(x)[0])
# # double['m_matches']=double['envelope_matches'].apply(lambda x : np.array(x)[1])
# double

In [60]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# double= double.apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [61]:
# double=double.apply(lambda x: f'"{x}"')
# double=double.apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# double[['one','two']]=double['envelope_matches'].str.split(",",expand=True,)
# double=double.apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# double

In [31]:
# envelope['envelope_matches']=envelope['envelope_matches'].str.get(0)
# envelope['envelope_matches']=envelope['envelope_matches'].apply(lambda x: x.lower())

In [32]:
# envelope[['width', 'length']]=envelope['envelope_matches'].str.split("x",expand=True,)

In [33]:
# envelope[envelope['width'].astype(str)=='77 34']

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, envelope_matches, matches, width, length]
Index: []

In [34]:
# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'77-34','34',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 500\/','12',str(measure)))
# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'12 500\/','12',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 2500\/','12',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 1000\/','12',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12\,100','12',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'10 100/','10',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'8 100/','8',str(measure)))

# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12100','12',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12\,100','12',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'10 100/','10',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'8 100/','8',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'16 100','16',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'19 500','19',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'77-34','34',str(measure)))


In [35]:
# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

In [36]:
# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

# envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# envelope['width']=envelope['width'].apply(lambda quote: quote.replace('"', ''))

# regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
# # envelope['length_new']=envelope['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
# envelope['width_new']=envelope['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)

# # envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_fraction(x))
# envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_fraction(x))

# # envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_frac(x))
# envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_frac(x))

In [37]:
# envelope['length_new']=envelope['length_new'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# envelope['width_new']=envelope['width_new'].apply(lambda quote: quote.replace('"', ''))

In [38]:
# TWOPLACES = Decimal(10) ** -2
# # envelope['length_new']=envelope['length_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
# envelope['width_new']=envelope['width_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
# envelope[curation_col]=envelope['width_new']

In [39]:
# envelope[curation_col]=envelope[curation_col].apply(lambda x: str(x)+' in')

In [40]:
# len(envelope)

5576

# Office Chairs

In [31]:
oc=height[height['buckets'].astype(str)=='Office Chairs']
# regex_pattern_general=r'(?i)(roll)|()'
regex_pattern=r'(?i)(?i)(\d+.?\d*\/?\d*.?\s?to\s?[a-z\s]{0,20}\d*.?\d*\/?\d*)|(seat.?width.?\s?\d+\"\s?\-?\s?\d*\s?\d*\/?\d*)|(\d\d\d?\d*\".{0,3}\d\d\.\d*\"\s?W)||()'              
oc['matches']=oc['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
oc_na=oc[oc['matches'].astype(str)=='[]']
oc=oc[oc['matches'].astype(str)!='[]']
oc_na[curation_col]=r'n/a'

In [33]:
oc

product_id external_id  \
5      693884212  5637546306   
6      434391923  5637246169   
206    434388111  5637241368   
600    536514438  5637521280   
601    536514437  5637521279   
602    536514435  5637521277   
819   1175424263  5637574404   
925    434496861  5637472688   
1021   434448804  5637400194   
1165   477457506  5637507344   
1410   434485464  5637149840   
1711   434418544  5637336353   
1835   434397221  5637259647   
2533   693884199  5637546310   
2589   536514418  5637521249   
2590   536514415  5637521246   
2752  1175502575  5637574842   
2753  1175502570  5637574837   
2754  1175502568  5637574835   
2762  1175502577  5637574844   
2818  1175408148  5637574327   
2819  1175405162  5637574321   
2863  1175461549  5637576131   
2870   693919090  5637546303   
2924   693888703  5637546307   
2931   522174678  5637444353   
2945  1175444821  5637576109   
2949  1175405166  5637574325   
2951   536514444  5637521286   
2959  1175424264  5637574405   
2980  1175444812  5637576096   
3162   434415990  5637330588   
3213  1175502580  5637574847   
3215  1175502578  5637574845   
3216  1175502571  5637574838   
3231   536514467  5637521262   
3278   693919091  5637546304   
3282   536514416  5637521247   
3337  1175394895  5637576168   
3352   795347336  5637560438   
3354  1175502573  5637574840   
3363   434437431  5637381422   
3369  1175424262  5637574403   
3401   536514431  5637521273   
3402  1175424254  5637574395   
3494  1175405161  5637574320   
3495   536512423  5637521241   
3496   693888729  5637546268   
3620  1175502581  5637574848   
3621  1175502567  5637574834   
3622  1175502564  5637574831   
3646  1175405167  5637574326   
4070  1175446419  5637574407   
4071  1175424261  5637574402   
4238   693884198  5637546309   
4239   693884211  5637546305   
4240   693919089  5637546302   
4313   536514432  5637521274   
4462   693888731  5637546270   
4503   434407460  5637305076   
4690   536514417  5637521248   
4699   536514434  5637521276   
4805   536514459  5637521254   
4865   434507725  5637486378   
4924  1175405160  5637574319   
4971   536514436  5637521278   
5069   434479715  5637456943   
5099  1175424258  5637574399   
5102   434418545  5637336354   
5154  1175398995  5637574849   
5170  1175502579  5637574846   
5175  1175405165  5637574324   
5489  1175424259  5637574400   
5490  1175424256  5637574397   
5492  1175424255  5637574396   
5534  1175502569  5637574836   
5535  1175502562  5637574829   
5541  1175405164  5637574323   
5589   536514433  5637521275   
5702   693884187  5637546272   
5703   693888730  5637546269   
5704   693888728  5637546267   
5956   693888704  5637546308   
5983   693884188  5637546273   
6291  1175502572  5637574839   
6292  1175398994  5637574828   
6300  1175405163  5637574322   
6406   434408270  5637305640   
6454  1175494673  5637575978   
6672  1175502587  5637574858   
6673  1175492659  5637574393   
6694   434505593  5637489442   
6702   693884186  5637546271   
6719  1175394894  5637576167   
6722   693919088  5637546301   
6724   434485465  5637149843   
6726   434449643  5637408677   
6843   434382753  5637181609   
7181   693884189  5637546274   
7231   434487430  5637164890   
8056   815178996  5637561693   
8486   434455907  5637416737   
8550   815178987  5637561684   
8632   434488912  5637159539   
8719  1175398993  5637574827   
8923  1175398996  5637574850   

                                                                                                                                            product_name  \
5                                              Scholar Craft 2Thrive Series Lab Chair, 23-33" Black Star Base Pneumatic Caster Chair, Primary Blue Shell   
6                        Alera Alera Etros Series Suspension Mesh Mid-Back Synchro Tilt Chair, Supports up to 275 lbs, Black Seat/Black Back, Black Base   
206                                                                                             HON Wave� Series 

In [32]:
len(oc_na) 

699

# Desk WorkBenches

In [34]:
desk=height[height['buckets'].astype(str)=='Desks/Workstations/Workbenches']
# regex_pattern_general=r'(?i)(roll)|()'
regex_pattern=r'(?i)(\d+.?w\s?x\s?30)|(\d+\.?\d*.?w\s?x\s?30)|(\d+\s?\-?\/?\d+.?w\s?x\s?30)|(\d+.?\s?x\s?30)|(\d+\.?\d*.?\s?x\s?30)|(\d+\-?\/?\d+.?\s?x\s?30)|(width)|(\"w)|(\d\d\.?\d\s?w)|(\d+\s?\d*\/\d*\s?w)|(\d+.?\s?x\s?\d*.?\d*\/\d+)|(\d+.?.?w?\s?x\s?\d+.?\d*.?)|(\d+.?\s?w\s?)|()'              
desk['matches']=desk['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
desk_na=desk[desk['matches'].astype(str)=='[]']
desk=desk[desk['matches'].astype(str)!='[]']
desk_na[curation_col]=r'n/a'

In [35]:
len(desk) 

639

In [36]:
desk['one']=desk['matches'][7:9].str.get(0)
desk['two']=desk['matches'][7:9].str.get(1)
desk['three']=desk['matches'][7:9].str.get(2)
desk['four']=desk['matches'][7:9].str.get(3)

In [37]:
desk['two'][0:1].astype(str)

37    nan
Name: two, dtype: object

In [38]:
desk[desk['two'].astype(str)!='nan']

product_id external_id  \
141  1175394879  5637576133   

                                                                       product_name  \
141  Flash Furniture Clifton Maple Computer Desk With Top And Lower Storage Shelves   

                                                                                                                                                                  long_desc  \
141  Clifton Maple Computer Desk With Top And Lower Storage Shelves•Contemporary style•Maple laminate finish•Top size: 28"W x 14"D•.5" thick top•Top shelf size: 28"W x 8"D   

     customer_id attributes                         buckets  bucket_id values  \
141           90       None  Desks/Workstations/Workbenches       7440   None   

                       matches         one          two  three  four  
141  [28"W x 8"D, 28"W x 14"D]  28"W x 8"D  28"W x 14"D    NaN   NaN

# Chairs

In [169]:
chair=height[height['buckets'].astype(str)=='Chairs']
regex_pattern=r'(?i)(\d+\.?\s?\-?\d*\/?\d*\"\s?w)|()'              
chair['matches']=chair['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
chair_height=chair[chair['matches'].astype(str)!='[]']

In [179]:
print(len(chair))
print(len(chair_height))

692
78


In [175]:
chair_three=chair[chair['matches'].astype(str)=='[]']
regex_pattern_three=r'(?i)(\d+.?w\s?x\s?30)|(\d+\.?\d*.?w\s?x\s?30)|(\d+\s?\-?\/?\d+.?w\s?x\s?30)|(\d+.?\s?x\s?30)|(\d+\.?\d*.?\s?x\s?30)|(\d+\-?\/?\d+.?\s?x\s?30)|(width)|(\"w)|(\d\d\.?\d\s?w)|(\d+\s?\d*\/\d*\s?w)|(\d+.?\s?x\s?\d*.?\d*\/\d+)|(\d+.?.?w?\s?x\s?\d+.?\d*.?)|(\d+.?\s?w\s?)|()'              
chair_three['matches']=chair_three['long_desc'].apply(lambda x: re_extract(regex_pattern_three, x))

In [176]:
chair_na=chair_three[chair_three['matches'].astype(str)=='[]']
chair_na[curation_col]=r'n/a'

In [177]:
print(len(chair_na)) 

584


# Drawers

In [186]:
drawer=height[height['buckets'].astype(str)=='Drawers']
print(len(drawer))
regex_pattern_drawer=r'(?i)(\d\d\.?\d*\s?w)|(\d+\s?\d*\/\d*\s?w)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?W)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|(\d+.?\d*\"\s?To\s?\d+")|(\d+\.?\s?\-?\d*\/?\d*"W)|(width)|()'                           
drawer['matches']=drawer['long_desc'].apply(lambda x: re_extract(regex_pattern_drawer, x))
drawer_na=drawer[drawer['matches'].astype(str)=='[]']
drawer_na[curation_col]=r'n/a'
print(len(drawer_na))

456
137


In [187]:
drawer_next=drawer[drawer['matches'].astype(str)!='[]']
regex_pattern_short=r'(?i)(\d\d\.?\d*\s?w)|(\d+\s?\-?\d*\/\d*\s?w)|(\d+\.?\s?\-?\d*\/?\d*"W)|()'                           
drawer_next['new_matches']=drawer_next['long_desc'].apply(lambda x: re_extract(regex_pattern_short, x))
drawer_short=drawer_next[drawer_next['new_matches'].astype(str)!='[]']
print(len(drawer_short))

293


In [192]:
drawer_short[drawer_short['width'].astype(str)=='500 16']

product_id external_id  \
14709  1175425699  5637577971   

                                                                               product_name  \
14709  Bush Business Furniture Office 500 16"W 3-Drawer File Cabinet, Storm Gray, Assembled   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    long_desc  \
14709  Office 500 16"W 3-Drawer File Cabinet, Storm Gray, Assembled•16"W x 20"D x 29"H 3-Drawer File Cabinet ships fully assembled for a stress-free set up•Drawers fit under Office 500 72"W x 36"D Executive Desk and 72"W x 24"D Credenza Desk (sold separately)•File drawer opens on smooth full-extension ball bearing slides for easy access to legal, letter and A4-size paperwork•Vertical filing cabinet also includes two box drawers for organizing small office supplies•Single lock secures two bottom drawers to protect sensitive documents and personal items   

       customer_id attributes  buckets  bucket_id values  \
14709           90       None  Drawers       7476   None   

                                matches                       new_matches  \
14709  [500 16"W, 72"W, 500 72"W, 16"W]  [500 16"W, 72"W, 500 72"W, 16"W]   

        width  
14709  500 16

In [209]:
drawer_short['width']=drawer_short['new_matches'].str.get(0)
drawer_short['width']=drawer_short['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
drawer_short['width']=drawer_short['width'].apply(lambda measure: re.sub(r'500 36','36',str(measure)))
drawer_short['width']=drawer_short['width'].apply(lambda measure: re.sub(r'500 16','16',str(measure)))
drawer_short['width']=drawer_short['width'].apply(lambda measure: re.sub(r"\'|,|:",'',str(measure)))
drawer_short['width']=drawer_short['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
drawer_short['width']=drawer_short['width'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
drawer_short['width'] = drawer_short['width'].apply(lambda x: remove_fraction(x))
drawer_short['width'] = drawer_short['width'].apply(lambda x: remove_frac(x))
drawer_short['width']=drawer_short['width'].astype(float)
TWOPLACES = Decimal(10) ** -2
drawer_short['width']=drawer_short['width'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
drawer_short[curation_col]=drawer_short['width']
drawer_short[curation_col]=drawer_short[curation_col].apply(lambda x: str(x)+' in')

In [196]:
drawer_three=drawer_next[drawer_next['new_matches'].astype(str)=='[]']
regex_pattern_three=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?W)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|()'
drawer_three['new_matches']=drawer_three['long_desc'].apply(lambda x: re_extract(regex_pattern_three, x))
drawer_three=drawer_three[drawer_three['new_matches'].astype(str)!='[]']

In [197]:
drawer_three['new']=drawer_three['new_matches'].str.get(0)
drawer_three[['length', 'width', 'height']]=drawer_three['new'].str.split("x",expand=True,)
drawer_three=drawer_three[(drawer_three['external_id'].astype(str)!='5637192984')&(drawer_three['external_id'].astype(str)!='5637192989')&(drawer_three['external_id'].astype(str)!='5637192983')&(drawer_three['external_id'].astype(str)!='5637192989')&(drawer_three['external_id'].astype(str)!='5637192990')]                      

In [198]:
drawer_three['width']=drawer_three['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
drawer_three['width']=drawer_three['width'].apply(lambda measure: re.sub(r"\'|,|:",'',str(measure)))
drawer_three['width']=drawer_three['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
drawer_three['width']=drawer_three['width'].replace('"', '').apply(lambda quote: quote.replace('"', ''))

In [199]:
regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
drawer_three['width_new']=drawer_three['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
drawer_three['width_new'] = drawer_three['width_new'].apply(lambda x: remove_fraction(x))

In [208]:
drawer_three['width_new']=drawer_three['width_new'].apply(lambda x: remove_fraction(x))
drawer_three['width_new']=drawer_three['width_new'].apply(lambda x: remove_frac(x))
drawer_three['width_new']=drawer_three['width_new'].astype(float)
TWOPLACES = Decimal(10) ** -2
drawer_three['width_new']=drawer_three['width_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
drawer_three[curation_col]=drawer_three['width_new']
drawer_three[curation_col]=drawer_three[curation_col].apply(lambda x: str(x)+' in')

In [210]:
print(len(drawer_short))

293


In [211]:
print(len(drawer_three))

22


In [212]:
drawer_three

product_id external_id  \
1054    477458063  5637506399   
1062    434387388  5637218263   
1125    434385116  5637216865   
1654    434385211  5637218880   
2526    434407631  5637306801   
4406    434485770  5637149753   
4407    434485769  5637149751   
8392    434451221  5637408078   
8393    434452026  5637408060   
8609    477457460  5637507300   
8790    477458060  5637506397   
9360    434451223  5637408080   
12686   477458064  5637506400   
12904   477458032  5637506376   
12956   477458033  5637506377   
13115   434451224  5637408081   
13236   474986760  5637506693   
13402   477458031  5637506375   
15071   434451222  5637408079   
15072   434452027  5637408061   
16979   477458061  5637506398   
17205   474986744  5637506682   

                                                                                               product_name  \
1054                              Lorell Revelance Mobile Pedestal, 15.8" x 19.9" x 22.9", Laminate, Walnut   
1062                  Safco Mira Series Wood Veneer Two-Drawer Lateral File, 34-1/2 x 24 x 29-1/2, Espresso   
1125                  Storex Two-Drawer Mobile Filing Cabinet With Metal Siding, 19 x 15 x 23, Silver/Black   
1654                  Safco Mira Series Wood Veneer Two-Drawer Lateral File, 34-1/2 x 24 x 29-1/2, Med Chry   
2526                    HON BL Laminate Three Drawer Pedestal File, 15 5/8 x 21 3/4 x 27 3/4, Medium Cherry   
4406                              HON BW Veneer Series Two-Drawer Lateral File, 36 x 24 x 29,Bourbon Cherry   
4407                     HON BW Veneer Series File/File Pedestal File, 15-5/8 x 22 x 27-3/4, Bourbon Cherry   
8392                                            HON Mobile Pedestal File, Box/Box/File, 15 x 20 x 28, Putty   
8393                                            HON Mobile Pedestal File, Box/Box/File, 15 x 20 x 28, Black   
8609                          Lorell 3-drawer Mobile Pedestal File, 15" x 19" x 26", Steel, Recycled, White   
8790            Lorell Relevance Series Box/File Mobile Pedestal, 15.8" x 19.9" x 22.9", Laminate, Mahogany   
9360                                               HON Mobile Pedestal File, File/File, 15 x 20 x 28, Black   
12686                      Lorell Mid-century Modern Box/File Mobile Pedestal, 19.8" x 16.5" x 23.5", White   
12904  Lorell Relevance Series Office File Cabinet Credenza, 29.5" x 22" x 23.1", Laminate, Charcoal/Silver   
12956                        Lorell Open Shelf File Cabinet Credenza, 29.5" x 22" x 23.1", Laminate, Walnut   
13115                                               HON Mobile Pedestal File, File/File, 15 x 20 x 28, Gray   
13236                                     HON 4-Drawer Lateral File, 36" x 19.3" x 53.3", Steel, Light Gray   
13402         Lorell Relevance Series Office File Cabinet Credenza, 29.5" x 22" x 23.1", Laminate, Mahogany   
15071                                              HON Mobile Pedestal File, File/File, 15 x 20 x 28, Putty   
15072                                            HON Mobile Pedestal File, Box/Box/File, 15 x 20 x 28, Gray   
16979           Lorell Relevance Series Box/File Mobile Pedestal, 15.8" x 19.9" x 22.9", Laminate, Charcoal   
17205                                     HON 2-Drawer Lateral File, 36" x 19.8" x 28.4", Steel, Light Gray   

                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
1054                                                                                              Revelance Mobile Pedestal, 15.8" x 19.9" x 22.9", Laminate, Walnut•Part of the Lorell Relevance Series•Mobile for easy access; 2 casters lock for reliable stability•Versatile storage wit

In [204]:
print(len(drawer_na))

137


In [205]:
print(len(chair_na)) 

584


In [143]:
chair_na

product_id external_id  \
13      492030428  5637508245   
15      434513539  5637490727   
16      434442195  5637389544   
17      434485502  5637171439   
18      434501924  5637476114   
...           ...         ...   
17625   815179205  5637541635   
17633   815177544  5637541618   
17641   815179215  5637541644   
17656   815176118  5637541626   
17658   815179200  5637541671   

                                                                                                        product_name  \
13                                                     Flash Furniture HERCULES Diplomat Series Chair, Leather, Gray   
15                     Flash Furniture HERCULES Series 661 lb. Capacity Black Full Back Stack Chair with Black Frame   
16                                                          HON Accommodate� Series Armless Guest Chair, Black Vinyl   
17                                                         Safco Uber Series Big & Tall Sled Base Guest Chair, Black   
18                                    Allsteel Lyric Multi-Purpose Chair, Polymer Back, Fixed Arms, Fixed Arms, Lava   
...                                                                                                              ...   
17625  Flash Furniture HERCULES Series Trapezoidal Back Stacking Banquet Chair in Navy Patterned Fabric - Gold Frame   
17633                   Flash Furniture HERCULES Series 21''W Stacking Church Chair in Plum Fabric - Gold Vein Frame   
17641                             Flash Furniture HERCULES Series Big & Tall 1000 lb. Rated Black Fabric Stack Chair   
17656  Flash Furniture HERCULES Series 21''W Church Chair in Navy Blue Fabric with Cup Book Rack - Silver Vein Frame   
17658                            Flash Furniture White LeatherSoft Executive Side Reception Chair with Mahogany Legs   

                                                                                                                                                                                                                                                                                                                                                   long_desc  \
13                                                                                                                                                                                                 HERCULES Diplomat Series Chair, Leather, Gray•Contemporary style•Gray LeatherSoft upholstery•Flared arms•Plush back and seat•Fixed back and seat cushions   
15                                                                                                                                                    HERCULES Series 661 lb. Capacity Black Full Back Stack Chair with Black Frame•Multipurpose stack chair•661 lb. weight capacity•Stack quantity: 25•Plastic back and seat•Ergonomically contoured design   
16                Accommodate� Series Armless Guest Chair, Black Vinyl•Lightweight, maneuverable frame is ideal for multi-purpose spaces, cafes and common areas.•Seat cushion is made of high-density molded foam that is designed to last longer.•Comfort contours in the seat back cradle the spine.•Waterfall seat edge promotes better leg circulation.   
17                                                                                                              Uber Series Big & Tall Sled Base Guest Chair, Black•Specifically designed for the tall and big person.•Extra-wide and deep seat.•Cushioned foam is rated for 24/7 use and chair is rated up to 500 lbs.•Reinforced base for added stability.   
18     Lyric Multi-Purpose Chair, Polymer Back, Fixed Arms, Fixed Arms, Lava•Perforated back provides ventilation and distributed support•Waterfall seat front reduces pressure points and the intuitive back design offers built-in lumbar support•Available with or without upholstered seat•Stack up to 10 chairs high•Warranted for users up to 300 lbs.   
...                                                      

In [64]:
len(desk_na) 

285

In [65]:
len(oc_na) 

693

In [66]:
len(envelope)

5578

In [57]:
err

NameError: name 'err' is not defined

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

In [ ]:
len(envelope)len(oc_na) len(desk_na) print(len(chair_na)) print(len(drawer_na)) print(len(drawer_three)) print(len(drawer_short))

# send to the folder for upload

In [85]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets- {attribute}-- not_tube- upload_Shawn.csv',index=False)

In [86]:
looks_good('Bed Bath & Beyond', attribute, df, not_tube)

In [91]:
def looks_good_na(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets-NA - {attribute}-- total_na_values upload_Shawn.csv',index=False)

In [92]:
looks_good_na('Bed Bath & Beyond', attribute, df, total_na_values)

In [ ]:
matches_side_table+matches_stool+matches_tree